<a href="https://colab.research.google.com/github/pdkary/VincentGANVogh/blob/main/Vinny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/pdkary/VincentGANVogh.git
!cd VincentGANVogh/ && git fetch
!cd VincentGANVogh/ && git pull
!pip install -r /content/VincentGANVogh/requirements.txt

import sys
sys.path.insert(0,'/content/VincentGANVogh')

In [ ]:
from GanConfig import GanShapeConfig,GanBuildingConfig,GanTrainingConfig
from GanTrainer import GanTrainer

from google.colab import drive
drive.mount('/content/drive')
 
gan_shape_config = GanShapeConfig(
    img_shape=(256,256,3),
    latent_size=4096,
    style_size=100,
    kernel_size=3,
    style_layer_size=128,
    style_layers=4,
    gen_layer_shapes=[(512,3),(256,2),(128,2),(64,2),(32,2),(16,2)],
    gen_layer_upsampling=[True,True,True,True,True,True],
    disc_layer_shapes=[(32,2),(64,2),(128,2),(256,3),(512,3),(512,3)],
    disc_dense_sizes=[4096,4096,1024],
    minibatch_size=12*256,
)
 
gan_building_config = GanBuildingConfig(
    relu_alpha=0.1,
    dropout_rate=0.5,
    batch_norm_momentum=0.8
)
 
gan_training_config = GanTrainingConfig(
    learning_rate=2e-4,
    disc_loss_function="binary_crossentropy",
    gen_loss_function="binary_crossentropy",
    use_latent_noise=False,
    gauss_factor=0.1,
    batch_size=12,
    preview_rows=3,
    preview_cols=4,
    data_path='/content/drive/MyDrive/Colab/VanGogh',
    image_type=".jpg",
    model_name='/GANVogh_generator_model_'
)


VGV = GanTrainer(gan_shape_config,gan_building_config,gan_training_config)

In [ ]:
#TRAINING
ERAS = 100
EPOCHS = 2500
BATCHES_PER_EPOCH = 1
PRINT_EVERY = 10
MOVING_AVERAGE_SIZE = 20

VGV.train_n_eras(ERAS,EPOCHS,BATCHES_PER_EPOCH,PRINT_EVERY,MOVING_AVERAGE_SIZE)